In [15]:
import os
import pandas as pd
import numpy

In [16]:
df = pd.read_csv('butler.csv')
df

,SEASON_YEAR,TEAM_ABBREVIATION,GAME_DATE,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,...,PCT_PTS_OFF_TOV_RANK,PCT_PTS_PAINT_RANK,PCT_AST_2PM_RANK,PCT_UAST_2PM_RANK,PCT_AST_3PM_RANK,PCT_UAST_3PM_RANK,PCT_AST_FGM_RANK,PCT_UAST_FGM_RANK,OPPONENT,HOME
0,2022-23,MIA,2022-10-19T00:00:00,L,36.566667,5,11,0.455,0,0,...,9,50,29,30,22,11,33,27,CHI,True
1,2022-23,MIA,2022-10-21T00:00:00,L,40.133333,9,17,0.529,0,1,...,2,8,26,38,22,11,31,33,BOS,True
2,2022-23,MIA,2022-10-22T00:00:00,W,34.833333,7,16,0.438,1,1,...,50,50,17,42,1,11,17,47,TOR,True
3,2022-23,MIA,2022-10-24T00:00:00,L,33.795000,8,15,0.533,2,3,...,15,56,8,55,1,11,4,58,TOR,True
4,2022-23,MIA,2022-10-26T00:00:00,W,27.966667,7,15,0.467,0,3,...,4,1,7,58,22,11,8,57,POR,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2022-23,MIA,2023-03-25T00:00:00,L,32.668333,6,13,0.462,1,2,...,45,23,52,12,22,1,56,8,BKN,True
60,2022-23,MIA,2023-03-29T00:00:00,L,34.683333,5,10,0.500,0,0,...,57,8,29,30,22,11,33,27,NYK,False
61,2022-23,MIA,2023-04-01T00:00:00,W,36.995000,12,16,0.750,1,3,...,43,53,44,21,1,11,41,22,DAL,True
62,2022-23,MIA,2023-04-04T00:00:00,W,35.483333,9,12,0.750,0,0,...,18,42,61,1,22,11,62,1,DET,False


In [17]:
df['next_pts'] = df['PTS'].shift(-1)

In [ ]:
df.dtypes[df.dtypes == "object"]
df.drop(["WL", "SEASON_YEAR"], axis=1)

In [44]:
df['team_code'] = df['TEAM_ABBREVIATION'].astype("category").cat.codes
df['opponent_code'] = df['OPPONENT'].astype("category").cat.codes
fulldf = df.copy()
df = df.dropna()

In [36]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

In [38]:
rr = Ridge(alpha=1)

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split, n_jobs=4)

In [50]:
removed_cols = ["next_pts", "SEASON_YEAR", "TEAM_ABBREVIATION", "OPPONENT", "GAME_DATE", "WL"]
selected_cols = df.columns[~df.columns.isin(removed_cols)]

In [51]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df.loc[:, selected_cols] = scaler.fit_transform(df[selected_cols])

In [53]:
df.describe()

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,PCT_AST_2PM_RANK,PCT_UAST_2PM_RANK,PCT_AST_3PM_RANK,PCT_UAST_3PM_RANK,PCT_AST_FGM_RANK,PCT_UAST_FGM_RANK,HOME,next_pts,team_code,opponent_code
count,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,...,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.0,63.000000
mean,0.578243,0.455026,0.383598,0.499769,0.138889,0.226757,0.254651,0.322291,0.378882,0.825317,...,0.504497,0.474175,0.736206,0.884127,0.498829,0.476190,0.523810,22.888889,0.0,0.467643
std,0.171212,0.217320,0.219680,0.234905,0.199686,0.238818,0.353440,0.177837,0.196380,0.205384,...,0.314455,0.293786,0.434658,0.301719,0.310314,0.292628,0.503444,6.942415,0.0,0.298658
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.0,0.000000
25%,0.528219,0.250000,0.222222,0.314751,0.000000,0.071429,0.000000,0.217391,0.260870,0.750000,...,0.258333,0.222222,0.404762,1.000000,0.237705,0.222222,0.000000,17.500000,0.0,0.192308
50%,0.590620,0.416667,0.388889,0.493997,0.000000,0.142857,0.000000,0.304348,0.347826,0.875000,...,0.466667,0.460317,1.000000,1.000000,0.524590,0.412698,1.000000,23.000000,0.0,0.461538
75%,0.659102,0.583333,0.500000,0.626072,0.250000,0.285714,0.500000,0.391304,0.478261,1.000000,...,0.733333,0.706349,1.000000,1.000000,0.770492,0.746032,1.000000,27.000000,0.0,0.711538
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,38.000000,0.0,1.000000


In [54]:
sfs.fit(df[selected_cols], df["next_pts"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=4)

In [56]:
predictors = list(selected_cols[sfs.get_support()])
predictors

['PFD',
 'FGA_RANK',
 'FTA_RANK',
 'BLKA_RANK',
 'DEF_RATING',
 'DREB_PCT',
 'NET_RATING_RANK',
 'sp_work_NET_RATING_RANK',
 'USG_PCT_RANK',
 'FGA_PG_RANK',
 'PCT_FG3M_RANK',
 'PCT_AST_2PM',
 'PCT_UAST_2PM',
 'PCT_AST_FGM',
 'PCT_UAST_FGM',
 'PCT_PTS_2PT_RANK',
 'PCT_PTS_2PT_MR_RANK',
 'PCT_PTS_3PT_RANK',
 'PCT_AST_2PM_RANK',
 'HOME']

In [65]:
def backtest(data, model, predictors, start=4, step=1):
    allpreds = []
    games = sorted(data["GAME_DATE"].unique())

    for i in range(start, len(games), step):
        current_game = games[i]
        train = data[data["GAME_DATE"] < current_game]
        test = data[data["GAME_DATE"] == current_game]

        model.fit(train[predictors], train['next_pts'])
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["next_pts"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        allpreds.append(combined)

    return pd.concat(allpreds)

In [66]:
predictions = backtest(df, rr, predictors)

In [67]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predictions["actual"], predictions["prediction"])

42.582920419493185

In [68]:
df["next_pts"].describe()

count    63.000000
mean     22.888889
std       6.942415
min       9.000000
25%      17.500000
50%      23.000000
75%      27.000000
max      38.000000
Name: next_pts, dtype: float64